In [ ]:
%matplotlib inline
from __future__ import print_function, division

import os, sys, glob
import collections 
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(context="poster", font_scale=1.5)

import ipywidgets as widgets
from ipywidgets import RadioButtons, fixed
from ipywidgets import interact, interactive, interact_manual

from IPython.display import display
import traitlets

## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        

from clustered_SNe import data_dir_default
from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   metallicity_solar
from clustered_SNe.analysis.parse import Overview, RunSummary, \
                                         extract_masses_momenta_raw

from clustered_SNe.analysis.visualize_helpers import plot_momentum, \
                                                     plot_momentum_scaling

from clustered_SNe.analysis.database_helpers import session, \
                                                    Simulation, \
                                                    Simulation_Inputs, \
                                                    Simulation_Status, \
                                                    extract_masses_momenta

In [ ]:
data_dir = data_dir_default

In [ ]:
metallicities_given_density = {}

for simulation in session.query(Simulation):
    if not simulation.background_density in metallicities_given_density:
        metallicities_given_density[simulation.background_density] = set()
    metallicities_given_density[simulation.background_density].add(simulation.metallicity)

densities = np.array(list(metallicities_given_density.keys()))
densities.sort()
log_densities = np.log10(densities / m_proton)
density_label_map = collections.OrderedDict() # so labels will be in order
for density in densities:
    log_density = np.log10(density / m_proton)
    density_label_map[str(round(log_density, 4))] = density

slice = {}
def extract_masses_momenta_wrapper(density):
    metallicities = np.array(list(metallicities_given_density[density]))
    metallicities.sort()
    metallicity_label_map = collections.OrderedDict()
    for metallicity in metallicities:
        log_metallicity = np.log10(metallicity / metallicity_solar)
        metallicity_label_map[str(round(log_metallicity, 4))] = metallicity
    extract_masses_momenta_wrapper.previous_widget.widget.close()
    v = interact_manual(extract_masses_momenta,
                        density = fixed(density),
                        metallicity = widgets.Dropdown(
                            options = metallicity_label_map,
                            description = 'Log10 (Z / Z_solar):'),
                       )
    extract_masses_momenta_wrapper.previous_widget = v
    def update_slice(widget):
        slice.clear()
        slice["masses"] = v.widget.result[0]
        slice["momenta"] = v.widget.result[1]
        slice["ids"] = v.widget.result[2]
    v.widget.children[-1].on_click(update_slice)
    return v
tmp = widgets.Box()
tmp.widget = widgets.Box()
extract_masses_momenta_wrapper.previous_widget = tmp

selected_run_widget = widgets.Text()
selected_run_widget.visible = False

In [ ]:
w = interact(extract_masses_momenta_wrapper,
             density= widgets.Dropdown(options=density_label_map,
                             description='Log10 (density / m_p):'))

In [ ]:
from bokeh import mpl
from bokeh.plotting import output_file, show, figure
from bokeh.models import HoverTool, TapTool, CustomJS
from bokeh.io import output_notebook
output_notebook()

from bokeh.models import ColumnDataSource
masses=slice["masses"] / M_solar
momenta=slice["momenta"] / slice["masses"] / (1000 * 100) + 1 # log scale can't handle 0
statuses = [session.query(Simulation_Status).get(id).status for id in slice["ids"]]
colors = []
for status in statuses:
    if status=="Complete":
        colors.append("green")
    elif (status=="Ready") or (status=="Running"):
        colors.append("orange")
    else:
        colors.append("red")
N_SNe = [session.query(Simulation).get(id).num_SNe for id in slice["ids"]]
source = ColumnDataSource(data=dict(masses=masses,
                                    momenta=momenta,
                                    ids=slice["ids"], 
                                    colors=colors,
                                    statuses=statuses,
                                    N_SNe=N_SNe))

p = figure(title="momentum scaling", 
           x_axis_type="log", 
           y_axis_type="log",
           x_range= [masses.min()/2, masses.max()*2],
           y_range= [10**3, momenta.max()*2],
           tools="reset,hover,tap, wheel_zoom")
p.xaxis.axis_label = "Masses [M_sol]"
p.yaxis.axis_label = "Momentum / Cluster Mass [km / s]"
p.scatter("masses","momenta", 
          source=source,
          size=10,
          legend="data", 
          color="colors")


hover = p.select(dict(type=HoverTool))
hover.tooltips = dict([
    ("Mass", "@masses"),
    ("Momentum / Cluster Mass", "@momenta"),
    ("ids", "@ids"),
    ("status", "@statuses"),
    ("N_SNe", "@N_SNe"),
])

source.callback = CustomJS(code="""
        var inds = cb_obj.get('selected')['1d'].indices;
        var d = cb_obj.get('data');
        for (i=0; i<inds.length; i++) {
            console.log("dataset id: " + d['ids'][inds[i]])
            // this is an unsecure hack, but it's safe enough if you are running your own code
            msg = "selected_run_widget.value = '" + d['ids'][inds[i]] + "'"
            console.log(msg);
            IPython.notebook.kernel.execute(msg);
            
            // copy id to clipboard
            var id_textarea = document.createElement("textarea")
            id_textarea.value = d['ids'][inds[i]]
            document.body.appendChild(id_textarea)
            id_textarea.select()
            document.execCommand('copy')
            document.body.removeChild(id_textarea)

    }
    """)


show(p)


In [ ]:
run_summary = RunSummary()
def show_momentum_evolution(data_dir, id):
    run_summary.replace_with(RunSummary(data_dir=data_dir, id=id))
    show_momentum_evolution.previous_widget.close()
    u = interactive(plot_momentum,
             run_summary=fixed(run_summary),
             x_axis = RadioButtons(options=["time", "checkpoints"]),
             clear_previous=fixed(True),
             distplot=fixed(True))
    display(u)
    show_momentum_evolution.previous_widget = u

show_momentum_evolution.previous_widget = widgets.Box()


w = widgets.interact_manual(show_momentum_evolution,
                     data_dir = fixed(data_dir),
                     id=widgets.Text())
l = traitlets.link((selected_run_widget, "value"), (w.widget.children[0], "value"))

In [ ]:
def set_status(id, status="Error"):
    assert(status in Simulation_Status.possible_statuses)
    session.query(Simulation_Status).get(id).status = status
    print("Id '"+ id + "' set to: ", status)
w_e = widgets.interact_manual(set_status, 
                              id = widgets.Text(), 
                              status = RadioButtons(options=list(Simulation_Status.possible_statuses)))
traitlets.link((selected_run_widget, "value"), (w_e.widget.children[0], "value"))

# Various plots I've sent to other people

In [ ]:
# left_data_dir = "../saved_runs/momentum_from_lower_density_background"
# left_id = "B1411D42-0C39-4A16-BD31-28CAD26B5C8D"

# left_data_dir = "../saved_runs/cluster_parameter_study"
# left_id = "4f206491-810e-4da9-a98d-19433981b68a"

left_data_dir = "../saved_runs/sample_mass_scaling"
left_id = "79EEEFCF-7546-4F0D-871E-58831E2BEB3A"

left_run_summary = RunSummary(data_dir=left_data_dir, id=left_id)


# right_data_dir = "../saved_runs/cluster_parameter_study"
right_data_dir = "../saved_runs/sample_mass_scaling/"
right_masses, right_momenta, right_ids = extract_masses_momenta_raw(right_data_dir, 
                                                                2.224587e-27, 
                                                                .02,
                                                                extract_at_last_SN = True)

with sns.axes_style("ticks"):
    with sns.plotting_context(context="paper", font_scale=2.4):
        f, (ax1, ax2) = plt.subplots(1,2, sharey=True)
        
        ax1.plot(left_run_summary.times / (10**6 * yr),
                 left_run_summary.momentum / (left_run_summary.overview.cluster_mass * 100 * 1000 * 1000),
                 linewidth=3)
        ax1.axes.set_xscale("linear")
#         ax1.axes.set_xbound(upper = 75)
        sns.distplot(left_run_summary.overview.SNe_times / (10**6 * yr), 
                     rug=True, hist=False, kde=False,ax=ax1, color="k", rug_kws={"linewidth":3})
        ax1.axes.set_xlabel("Time [Myr]")
        ax1.axes.set_ylabel("Momentum / M$_\mathrm{cluster}$ \n [10$^3$km s$^{-1}$]")
        ax1.xaxis.set_ticks(np.array([0,25,50,75,100]))
        
        
        ax2.scatter(right_masses / M_solar, right_momenta / (right_masses * 100 * 1000 * 1000), s=3*20)
        ax2.axes.set_xscale("log")
        ax2.axes.set_xlabel("M$_{\mathrm{cluster}} $ [M$_\odot$]")
        ax2.xaxis.set_ticks(right_masses / M_solar)

        
        plt.ylim(ymin=0)

        plt.tight_layout(w_pad=0)
        plt.savefig("plots/NSF_momentum.eps", pad_inches=.1, bbox_inches="tight")

In [ ]:
_data_dir = "../saved_runs/sample_mass_scaling_high_density/"
_masses, _momenta, _ids = extract_masses_momenta(_data_dir, 1.33 * m_proton, .02, 
                                                 extract_at_last_SN = True)

mask = _momenta >= _momenta[_masses == M_solar * 10**2].max()
_masses = _masses[mask]
_momenta = _momenta[mask]
_ids = _ids[mask]

# data_dir_tmp = "../saved_runs/sample_mass_scaling_high_density/"
# masses_tmp, momenta_tmp, ids_tmp = extract_masses_momenta(data_dir_tmp, 1.33 * m_proton, .02)

i = -1
with sns.axes_style("ticks"):
    plt.scatter(_masses/M_solar, _momenta / (_masses) / (1e5), s=40)
    plt.xscale("log")
    plt.yscale("log")
    plt.plot(_masses/M_solar, (_momenta[i]/(_masses[i]) / (1e5))*(_masses / _masses[i])**(-.2),
             label=r"$p \sim N^{0.8}$")
    plt.plot(_masses/M_solar, (_momenta[i]/(_masses[i]) / (1e5))*(_masses / _masses[i])**((23/25)-1),
             label=r"$p \sim N^{0.8 + 0.12}$", linestyle="--")
    plt.legend(loc="best")
    plt.ylabel("Momentum / Cluster mass [km/s]")
    plt.xlabel("Cluster mass [M_solar]")
    plt.tight_layout()
    filename = "momentum_scaling_comparison_high_density"
    plt.savefig(filename + ".eps")
    plt.savefig(filename + ".png")
    plt.savefig(filename + ".pdf")

    plt.figure()
    plt.scatter(_masses/M_solar, _momenta)
    plt.xscale("log")
    plt.yscale("log")
    plt.plot(_masses/M_solar, (_momenta[i])*(_masses / _masses[i])**(.8), label="simple")
    plt.plot(_masses/M_solar, (_momenta[i])*(_masses / _masses[i])**(23/25), label="full")
    plt.legend(loc="best")

In [ ]:
c = RadioButtons(options=list(Simulation_Status.possible_statuses))
c


In [ ]:


i = 4
with sns.axes_style("ticks"):
    plt.scatter(right_masses/M_solar, right_momenta / (right_masses) / (1e5), s=40)
    plt.xscale("log")
    plt.yscale("log")
    plt.plot(right_masses/M_solar, (right_momenta[i]/(right_masses[i]) / (1e5))*(right_masses / right_masses[i])**(-.2),
             label=r"$p \sim N^{0.8}$")
    plt.plot(right_masses/M_solar, (right_momenta[i]/(right_masses[i]) / (1e5))*(right_masses / right_masses[i])**((23/25)-1),
             label=r"$p \sim N^{0.8 + 0.12}$", linestyle="--")
    plt.legend(loc="best")
    plt.ylabel("Momentum / Cluster mass [km/s]")
    plt.xlabel("Cluster mass [M_solar]")
    plt.tight_layout()
    filename = "momentum_scaling_comparison"
    plt.savefig(filename + ".eps")
    plt.savefig(filename + ".png")
    plt.savefig(filename + ".pdf")

    plt.figure()
    plt.scatter(right_masses/M_solar, right_momenta)
    plt.xscale("log")
    plt.yscale("log")
    plt.plot(right_masses/M_solar, (right_momenta[i])*(right_masses / right_masses[i])**(.8), label="simple")
    plt.plot(right_masses/M_solar, (right_momenta[i])*(right_masses / right_masses[i])**(23/25), label="full")
    plt.legend(loc="best")

In [ ]:
two_SNe_data_dir = "../saved_runs/cluster_parameter_study"
two_SNe_id = "4f206491-810e-4da9-a98d-19433981b68a"
two_SNe_run_summary = RunSummary(data_dir=two_SNe_data_dir, id=two_SNe_id)

three_SNe_data_dir = "../saved_runs/sample_mass_scaling"
three_SNe_id = "79EEEFCF-7546-4F0D-871E-58831E2BEB3A"
three_SNe_run_summary = RunSummary(data_dir=three_SNe_data_dir, id=three_SNe_id)

ten_SNe_data_dir = "../saved_runs/cluster_parameter_study"
ten_SNe_id = "4bcff692-7d1e-45d0-ad77-81d3346db3a7"
ten_SNe_run_summary = RunSummary(data_dir=ten_SNe_data_dir, id=ten_SNe_id)

hundred_SNe_data_dir = "../saved_runs/cluster_parameter_study"
hundred_SNe_id = "b38e5d02-eeaa-4247-b00f-ab90da9caa30"
hundred_SNe_run_summary = RunSummary(data_dir=hundred_SNe_data_dir, id=hundred_SNe_id)

thousand_SNe_data_dir = "../saved_runs/cluster_parameter_study"
thousand_SNe_id = "90336918-56c8-40c4-89cd-a3ae7cf32ceb"
thousand_SNe_run_summary = RunSummary(data_dir=thousand_SNe_data_dir, id=thousand_SNe_id)


scaling_data_dir = "../saved_runs/sample_mass_scaling/"
scaling_masses, scaling_momenta, scaling_ids = extract_masses_momenta(scaling_data_dir, 2.224587e-27, .02)



In [ ]:
def evolution_plotter(tmp_run_summary):
    plt.plot(tmp_run_summary.times / (10**6 * yr), tm
             tmp_run_summary.momentum / (tmp_run_summary.overview.cluster_mass * 100 * 1000 * 1000),
             linewidth=3)
    plt.xscale("linear")
#         plt.xlim(xmax=75)
    sns.distplot(tmp_run_summary.overview.SNe_times / (10**6 * yr), 
                 rug=True, hist=False, kde=False, color="k", rug_kws={"linewidth":3})
    plt.xlabel("Time [Myr]")
    plt.ylabel("Momentum / M$_\mathrm{cluster}$ \n [10$^3$km s$^{-1}$]")
#         plt.gca().xaxis.set_ticks(np.array([0,20,40, 60]))

    plt.ylim(ymin=0)

    plt.tight_layout(w_pad=0)

with sns.axes_style("ticks"):
    with sns.plotting_context(context="talk", font_scale=2):
        
        evolution_plotter(two_SNe_run_summary)

        plot_name = "plots/avishai/2_SNe"
        plt.savefig(plot_name + ".eps")
        plt.savefig(plot_name + ".png")
        plt.savefig(plot_name + ".pdf")
        plt.close()

        ########
        
        evolution_plotter(three_SNe_run_summary)

        plot_name = "plots/avishai/3_SNe"
        plt.savefig(plot_name + ".eps")
        plt.savefig(plot_name + ".png")
        plt.savefig(plot_name + ".pdf")
        plt.close()

        ########      

        evolution_plotter(ten_SNe_run_summary)

        plot_name = "plots/avishai/10_SNe"
        plt.savefig(plot_name + ".eps")
        plt.savefig(plot_name + ".png")
        plt.savefig(plot_name + ".pdf")
        plt.close()

        ########
        
        evolution_plotter(hundred_SNe_run_summary)

        plot_name = "plots/avishai/100_SNe"
        plt.savefig(plot_name + ".eps")
        plt.savefig(plot_name + ".png")
        plt.savefig(plot_name + ".pdf")
        plt.close()

        ########
        
        evolution_plotter(thousand_SNe_run_summary)

        plot_name = "plots/avishai/1000_SNe"
        plt.savefig(plot_name + ".eps")
        plt.savefig(plot_name + ".png")
        plt.savefig(plot_name + ".pdf")
        plt.close()
        
        ########
        
        plt.plot(scaling_masses / M_solar, scaling_momenta / (scaling_masses * 100 * 1000 * 1000), marker="o")
        plt.xscale("log")
#         plt.yscale("log")
        plt.xlabel("M$_{\mathrm{cluster}} $ [M$_\odot$]")
        plt.ylabel("Momentum / M$_\mathrm{cluster}$ \n [10$^3$km s$^{-1}$]")

        plt.gca().xaxis.set_ticks(scaling_masses / M_solar)
        
        plt.xlim( (scaling_masses / M_solar).min()/2, (scaling_masses / M_solar).max()*2 )
        plt.ylim(ymin=0, ymax=25)

        plt.tight_layout(w_pad=0)

        plot_name = "plots/avishai/momentum_scaling"
        plt.savefig(plot_name + ".eps")
        plt.savefig(plot_name + ".png")
        plt.savefig(plot_name + ".pdf")
        plt.close()

In [ ]:
np.log(scaling_momenta[3]/scaling_momenta[1]) / np.log(scaling_masses[3]/scaling_masses[1]) - 1

In [ ]:
t = two_SNe_run_summary.overview
print( "SNe masses: ", t.SNe_initial_mass / M_solar)
print( "SNe times: ", t.SNe_times / (1e6 *yr))

In [ ]:
t = ten_SNe_run_summary.overview
print( "SNe masses: ", t.SNe_initial_mass / M_solar)
print( "SNe times: ", t.SNe_times / (1e6 *yr))

In [ ]:
t = hundred_SNe_run_summary.overview
print( "SNe masses: ", t.SNe_initial_mass / M_solar)
print( "SNe times: ", t.SNe_times / (1e6 *yr))

In [ ]:
t = thousand_SNe_run_summary.overview
print( "SNe masses: ", t.SNe_initial_mass / M_solar)
print( "SNe times: ", t.SNe_times / (1e6 *yr))
plt.plot()